In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

## General Glimpse

In [2]:
data = pd.read_csv('fifa21_male2.csv', sep=",")
data.head()

,ID,Name,Age,OVA,Nationality,Club,BOV,BP,Position,Player Photo,Club Logo,Flag Photo,POT,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,Gender
0,2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,https://cdn.sofifa.com/players/000/002/16_120.png,https://cdn.sofifa.com/teams/55/light_60.png,https://cdn.sofifa.com/flags/it.png,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",NaN,€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0,Male
1,16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,https://cdn.sofifa.com/players/000/016/19_120.png,https://cdn.sofifa.com/teams/2013/light_60.png,https://cdn.sofifa.com/flags/es.png,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",NaN,€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1,Male
2,27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,https://cdn.sofifa.com/players/000/027/16_120.png,https://cdn.sofifa.com/teams/1800/light_60.png,https://cdn.sofifa.com/flags/gb-eng.png,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",NaN,€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0,Male
3,36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,https://cdn.sofifa.com/players/000/036/09_120.png,https://cdn.sofifa.com/teams/106/light_60.png,https://cdn.sofifa.com/flags/tt.png,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0,Male
4,41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,https://cdn.sofifa.com/players/000/041/20_120.png,https://cdn.sofifa.com/teams/101146/light_60.png,https://cdn.sofifa.com/flags/es.png,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",NaN,€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3,Male


In [3]:
data.shape

(17125, 107)

## Split in 4 main groups: GK, DF, MF and AT

In [4]:
data["BP"].unique()

array(['LWB', 'CM', 'CAM', 'ST', 'RW', 'CB', 'CF', 'GK', 'CDM', 'RWB',
       'LB', 'LM', 'RM', 'RB', 'LW'], dtype=object)

In [5]:
#Goalkeeper
GK = data[["ID", "Name", "Age", "Club", "OVA", "BP", "Value","Goalkeeping", "GK Diving", "GK Handling", "GK Kicking", "GK Positioning", "GK Reflexes", "Total Stats"]]
GK = GK.loc[GK['BP'] == "GK"]
GK = GK.drop(['BP'],axis=1)
GK.head()

,ID,Name,Age,Club,OVA,Value,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats
14,488,O. Kahn,38,FC Bayern München,87,€0,346,83,88,62,92,83,1196
18,684,M. Schwarzer,42,Leicester City,73,€500K,357,70,73,67,78,69,1111
19,805,J. Lehmann,39,VfB Stuttgart,79,€0,323,77,82,75,88,76,1224
20,1001,G. Király,39,Hungary,69,€0,340,71,70,60,71,68,1084
26,1179,G. Buffon,42,Juventus,82,€2.2M,396,77,76,74,91,78,1292


In [6]:
GK.shape

(1576, 13)

In [7]:
#Defender
DF = data[["ID", "Name", "Age", "Club", "OVA", "BP", "Value","Heading Accuracy","Short Passing","Long Passing","Ball Control","Movement","Acceleration","Sprint Speed","Agility","Reactions","Balance","Jumping","Stamina","Strength","Mentality","Aggression","Interceptions","Positioning","Defending","Marking","Standing Tackle","Sliding Tackle","Total Stats"]]
DF = DF.loc[DF['BP'].isin(["LWB","CB","RWB","LB","RB"])]          
DF = DF.drop(['BP'],axis=1)
DF.head()

,ID,Name,Age,Club,OVA,Value,Heading Accuracy,Short Passing,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Jumping,Stamina,Strength,Mentality,Aggression,Interceptions,Positioning,Defending,Marking,Standing Tackle,Sliding Tackle,Total Stats
0,2,G. Pasquale,33,Udinese,69,€625K,59,71,69,71,347,68,74,68.0,69,68.0,68.0,69,68,320,72,69.0,63.0,208,70,69,69.0,1929
8,244,G. Neville B,35,Manchester United,76,€0,75,71,61,64,324,64,70,51.0,72,67.0,70.0,65,74,319,87,85.0,45.0,242,78,81,83.0,1774
10,249,P. Neville,35,Everton,74,€120K,69,74,72,72,321,52,51,65.0,83,70.0,71.0,61,76,335,78,83.0,48.0,224,77,75,72.0,1868
13,388,S. Campbell,35,Newcastle United,75,€0,81,54,44,55,321,61,68,44.0,68,80.0,85.0,64,87,296,84,77.0,41.0,222,72,77,73.0,1581
15,496,D. Hamann,36,Milton Keynes Dons,61,€0,62,76,64,64,212,40,44,28.0,30,70.0,83.0,28,82,344,74,75.0,51.0,198,70,72,56.0,1698


In [8]:
DF.shape

(5571, 28)

In [9]:
#Midfielder
MF = data[["ID", "Name","Nationality", "Age", "Club", "OVA", "BP", "Value","Attacking","Crossing","Finishing","Heading Accuracy","Short Passing","Volleys","Skill","Dribbling","Curve","FK Accuracy","Long Passing","Ball Control","Movement","Acceleration","Sprint Speed","Agility","Reactions","Balance","Power","Shot Power","Jumping","Stamina","Strength","Long Shots","Mentality","Aggression","Interceptions","Positioning","Vision","Penalties","Defending","Marking","Standing Tackle","Sliding Tackle","Total Stats"]]
MF = MF.loc[MF['BP'].isin(["CDM","CM","CAM","RM","LM"])]
MF = MF.drop(['BP'],axis=1)
MF.head()

,ID,Name,Nationality,Age,Club,OVA,Value,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Defending,Marking,Standing Tackle,Sliding Tackle,Total Stats
1,16,Luis García,Spain,37,KAS Eupen,71,€600K,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,153,70,43,40.0,1906
2,27,J. Cole,England,33,Coventry City,71,€1.1M,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,99,35,34,30.0,1770
4,41,Iniesta,Spain,36,Vissel Kobe,81,€5.5M,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,181,68,57,56.0,2014
6,80,E. Belözoğlu,Turkey,38,Fenerbahçe SK,77,€2.8M,342,80,66,43,84,69.0,406,77,83.0,80,85,81,308,54,36,67.0,70,81.0,340,77,72.0,51,64,76,384,87,62.0,65.0,85.0,85,190,65,65,60.0,2017
7,241,R. Giggs,Wales,39,Manchester United,78,€1K,380,90,70,60,85,75.0,402,77,87.0,78,81,79,337,59,60,72.0,76,70.0,310,71,59.0,48,61,71,358,60,47.0,78.0,83.0,90,120,26,43,51.0,1939


In [10]:
MF.shape

(6691, 42)

In [11]:
#Attacker
AT = data[["ID", "Name", "Age", "Club", "OVA", "BP", "Value", "Attacking", "Crossing","Finishing","Heading Accuracy","Short Passing","Volleys","Skill","Dribbling","FK Accuracy", "Acceleration", "Sprint Speed","Power","Shot Power","Stamina","Strength","Penalties","Long Shots","Mentality","Total Stats"]]
AT = AT.loc[AT['BP'].isin(["ST","CF","LW","RW"])]#selecting positions that apply to an attacker
AT = AT.drop(['BP'],axis=1)#dropping BP because we dont need it anymore
AT.head()

,ID,Name,Age,Club,OVA,Value,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,FK Accuracy,Acceleration,Sprint Speed,Power,Shot Power,Stamina,Strength,Penalties,Long Shots,Mentality,Total Stats
3,36,D. Yorke,36,Sunderland,68,€0,264,54,70,60,80,NaN,255,68,46,59,62,239,63,51,66,70,59,271,1348
5,61,D. Odonkor,27,Alemannia Aachen,66,€725K,271,61,53,42,58,57.0,276,67,42,90,91,315,71,64,61,46,49,256,1649
12,330,R. Keane,35,LA Galaxy,80,€5.5M,380,68,84,63,81,84.0,374,82,63,69,68,363,81,72,68,75,80,343,1974
16,570,A. Okocha,33,Hull City,73,€0,247,72,60,40,75,NaN,334,90,83,68,64,252,74,41,57,84,80,249,1405
17,670,Zulu,26,Pohang Steelers,66,€0,305,43,66,73,62,61.0,269,61,47,72,76,351,56,71,80,52,65,261,1704


In [12]:
AT.shape

(3287, 25)

## Data Cleaning


#### Time for Defenders

In [13]:
DF.isnull().sum()

ID                   0
Name                 0
Age                  0
Club                 5
OVA                  0
Value                0
Heading Accuracy     0
Short Passing        0
Long Passing         0
Ball Control         0
Movement             0
Acceleration         0
Sprint Speed         0
Agility             13
Reactions            0
Balance             13
Jumping             13
Stamina              0
Strength             0
Mentality            0
Aggression           0
Interceptions        2
Positioning          2
Defending            0
Marking              0
Standing Tackle      0
Sliding Tackle      13
Total Stats          0
dtype: int64

In [14]:
missing_jumping = DF[DF['Jumping'].isnull()].index.tolist()
missing_balance = DF[DF['Balance'].isnull()].index.tolist()
if missing_jumping == missing_balance:
    print('They are same')
else:
    print('They are different')

They are same


In [15]:
DF = DF[DF['Club'].notna()]
DF = DF[DF['Agility'].notna()]
DF.shape

(5554, 28)

In [16]:
DF.isnull().sum()

ID                  0
Name                0
Age                 0
Club                0
OVA                 0
Value               0
Heading Accuracy    0
Short Passing       0
Long Passing        0
Ball Control        0
Movement            0
Acceleration        0
Sprint Speed        0
Agility             0
Reactions           0
Balance             0
Jumping             0
Stamina             0
Strength            0
Mentality           0
Aggression          0
Interceptions       0
Positioning         0
Defending           0
Marking             0
Standing Tackle     0
Sliding Tackle      0
Total Stats         0
dtype: int64

#### Time for Midfielders

In [17]:
MF.isnull().sum()

ID                  0
Name                0
Nationality         0
Age                 0
Club                8
OVA                 0
Value               0
Attacking           0
Crossing            0
Finishing           0
Heading Accuracy    0
Short Passing       0
Volleys             9
Skill               0
Dribbling           0
Curve               9
FK Accuracy         0
Long Passing        0
Ball Control        0
Movement            0
Acceleration        0
Sprint Speed        0
Agility             9
Reactions           0
Balance             9
Power               0
Shot Power          0
Jumping             9
Stamina             0
Strength            0
Long Shots          0
Mentality           0
Aggression          0
Interceptions       1
Positioning         1
Vision              9
Penalties           0
Defending           0
Marking             0
Standing Tackle     0
Sliding Tackle      9
Total Stats         0
dtype: int64

In [18]:
missing_agility = MF[MF['Agility'].isnull()].index.tolist()
missing_balance = MF[MF['Balance'].isnull()].index.tolist()
if missing_agility == missing_balance:
    print('They are same')
else:
    print('They are different')

They are same


In [19]:
MF = MF[MF['Club'].notna()]
MF = MF[MF['Agility'].notna()]
MF.shape

(6674, 42)

In [20]:
MF.isnull().sum()

ID                  0
Name                0
Nationality         0
Age                 0
Club                0
OVA                 0
Value               0
Attacking           0
Crossing            0
Finishing           0
Heading Accuracy    0
Short Passing       0
Volleys             0
Skill               0
Dribbling           0
Curve               0
FK Accuracy         0
Long Passing        0
Ball Control        0
Movement            0
Acceleration        0
Sprint Speed        0
Agility             0
Reactions           0
Balance             0
Power               0
Shot Power          0
Jumping             0
Stamina             0
Strength            0
Long Shots          0
Mentality           0
Aggression          0
Interceptions       0
Positioning         0
Vision              0
Penalties           0
Defending           0
Marking             0
Standing Tackle     0
Sliding Tackle      0
Total Stats         0
dtype: int64

In [21]:
def value_to_int(df_value):
    try:
        value = float(df_value[1:-1])
        suffix = df_value[-1:]

        if suffix == 'M':
            value = value * 1000000
        elif suffix == 'K':
            value = value * 1000
    except ValueError:
        value = 0
    return value

DF['Value'] = DF['Value'].apply(value_to_int)
MF['Value'] = MF['Value'].apply(value_to_int)

In [22]:
MF_features=MF[["Attacking","Crossing","Finishing","Heading Accuracy","Short Passing","Volleys","Skill","Dribbling","Curve","FK Accuracy","Long Passing","Ball Control","Movement","Acceleration","Sprint Speed","Agility","Reactions","Balance","Power","Shot Power","Jumping","Stamina","Strength","Long Shots","Mentality","Aggression","Interceptions","Positioning","Vision","Penalties","Defending","Marking","Standing Tackle","Sliding Tackle"]]
MF_features.head()

,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Defending,Marking,Standing Tackle,Sliding Tackle
1,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,153,70,43,40.0
2,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,99,35,34,30.0
4,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,181,68,57,56.0
6,342,80,66,43,84,69.0,406,77,83.0,80,85,81,308,54,36,67.0,70,81.0,340,77,72.0,51,64,76,384,87,62.0,65.0,85.0,85,190,65,65,60.0
7,380,90,70,60,85,75.0,402,77,87.0,78,81,79,337,59,60,72.0,76,70.0,310,71,59.0,48,61,71,358,60,47.0,78.0,83.0,90,120,26,43,51.0


In [23]:
DF_features=DF[["Heading Accuracy","Short Passing","Long Passing","Ball Control","Movement","Acceleration","Sprint Speed","Agility","Reactions","Balance","Jumping","Stamina","Strength","Mentality","Aggression","Interceptions","Positioning","Defending","Marking","Standing Tackle","Sliding Tackle"]]
DF_features.head()

,Heading Accuracy,Short Passing,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Jumping,Stamina,Strength,Mentality,Aggression,Interceptions,Positioning,Defending,Marking,Standing Tackle,Sliding Tackle
0,59,71,69,71,347,68,74,68.0,69,68.0,68.0,69,68,320,72,69.0,63.0,208,70,69,69.0
8,75,71,61,64,324,64,70,51.0,72,67.0,70.0,65,74,319,87,85.0,45.0,242,78,81,83.0
10,69,74,72,72,321,52,51,65.0,83,70.0,71.0,61,76,335,78,83.0,48.0,224,77,75,72.0
13,81,54,44,55,321,61,68,44.0,68,80.0,85.0,64,87,296,84,77.0,41.0,222,72,77,73.0
15,62,76,64,64,212,40,44,28.0,30,70.0,83.0,28,82,344,74,75.0,51.0,198,70,72,56.0
